<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.dash table中 被打勾的項目（row) 更換其背景色

In [ ]:
import dash
from dash import dash_table, dcc, html
import pandas as pd

app = dash.Dash(__name__)

# 建立 DataFrame
df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['A', 'B', 'C', 'D', 'E']
})

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=5,
        row_selectable="multi",  # 允許多選
        selected_rows=[],  # 預設無選取,是一個列表（list），裡面存的是 被勾選的行的"索引"）。
        style_data_conditional=[
            {
                'if': {'row_index': 'selected'},  # 當 row 被勾選時
                'backgroundColor': 'lightblue',  # 設定背景顏色
                'color': 'black'  # 設定文字顏色
            }
        ]
    )
])



## 2. 想知道選取的row_index，其data

In [ ]:
import dash
from dash import dash_table, dcc, html, callback, Input, Output, State
import pandas as pd

app = dash.Dash(__name__)

# 建立 DataFrame
df = pd.DataFrame({
    'id': [101, 102, 103, 104, 105],
    'name': ['A', 'B', 'C', 'D', 'E'],
    'value': [10, 20, 30, 40, 50]
})

app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=5,
        row_selectable="multi",  # 允許多選
        selected_rows=[],
    ),
    html.Button("下載 Excel", id="download-btn", n_clicks=0),
    dcc.Download(id="download-dataframe-xlsx")
])

@callback(
    Output("download-dataframe-xlsx", "data"),
    Input("download-btn", "n_clicks"),
    State("table", "selected_rows"),
    prevent_initial_call=True
)
def download_selected_data(n_clicks, selected_rows):
    # 取得被勾選的行的完整數據 (!!!!!)
    selected_data = df.iloc[selected_rows]

    # 如果沒有選擇任何行，就不下載
    if selected_data.empty:
        return dash.no_update

    # 轉換成 Excel 檔案並下載
    return dcc.send_data_frame(selected_data.to_excel, "selected_data.xlsx", index=False)



## 3. figure放置

### 3-1. /assets/ 資料夾

1. 解析 Dash Table 中的資料：讀取每一行的 date, base, stat, chart_name, query_start, query_end 欄位。
2. 建立圖片路徑：根據這些欄位來生成圖片的路徑。
3. 排除重複圖片：使用 set 來避免重複顯示相同圖片。
4. 動態顯示圖片：將所有圖片顯示在頁面上。

* 這個範例假設圖片存放於 /assets/ 資料夾下，這樣 Dash 頁面可以正確顯示圖片。
* 如果圖片不在 assets/ 資料夾內，則需要透過 Flask 提供靜態檔案（如 send_from_directory）來處理圖片的路徑。

In [ ]:
import os
import dash
from dash import dcc, html, dash_table
import pandas as pd

# 假設你的資料表格長這樣：
data = [
    {'date': '2025-01-01', 'base': 'USD', 'stat': 'Open', 'chart_name': 'Chart1', 'query_start': '2025-01-01', 'query_end': '2025-01-02'},
    {'date': '2025-01-01', 'base': 'USD', 'stat': 'Close', 'chart_name': 'Chart1', 'query_start': '2025-01-01', 'query_end': '2025-01-02'},
    {'date': '2025-01-02', 'base': 'EUR', 'stat': 'Open', 'chart_name': 'Chart2', 'query_start': '2025-01-02', 'query_end': '2025-01-03'},
]

df = pd.DataFrame(data)

# 假設圖片檔案儲存在 ../data_source 資料夾內
IMAGE_FOLDER = '../data_source'

# 函數：生成圖片路徑
def generate_image_path(row):
    date = row['date']
    base = row['base']
    stat = row['stat']
    chart_name = row['chart_name']
    query_start = row['query_start']
    query_end = row['query_end']

    # 圖片路徑格式
    return f"/assets/{date}/{base}/figure/{chart_name}___{base}___{stat}___{query_start}___{query_end}.png"

# 取得表格中所有圖片路徑
image_paths = set()  # 使用 set 來排除重複圖片
for _, row in df.iterrows():
    image_paths.add(generate_image_path(row))

# 建立 Dash 應用程式
app = dash.Dash(__name__)

# 生成圖片顯示
app.layout = html.Div([
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": col, "id": col} for col in df.columns
        ],
        data=df.to_dict('records')
    ),
    html.Div([
        html.H3("所有相關圖片："),
        html.Div([html.Img(src=img_path, style={'width': '50%'}) for img_path in image_paths])
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)


### 3-2. Flask ()

如果你的圖片存放在其他資料夾，例如 ../data_source/，你需要用 Flask 的 send_from_directory 來提供靜態圖片。

In [ ]:
import os
import dash
from dash import dcc, html, dash_table
import pandas as pd
from flask import Flask, send_from_directory

# 初始化 Flask 伺服器
server = Flask(__name__) # __name__ 是一個特殊變數，它代表當前腳本的名稱。
# 當你執行一個 Python 腳本時，__name__ 的值會是 "__main__"
app = dash.Dash(__name__, server=server)

# 設定圖片資料夾
IMAGE_FOLDER = "../data_source"

# Flask 路由：當前端請求 `/images/xxx.png`，從資料夾提供圖片
@server.route("/images/<path:image_name>")
def serve_image(image_name):
    return send_from_directory(IMAGE_FOLDER, image_name)

# 假設這是你的 Dash Table 資料
data = [
    {'date': '2025-01-01', 'base': 'USD', 'stat': 'Open', 'chart_name': 'Chart1', 'query_start': '2025-01-01', 'query_end': '2025-01-02'},
    {'date': '2025-01-01', 'base': 'USD', 'stat': 'Close', 'chart_name': 'Chart1', 'query_start': '2025-01-01', 'query_end': '2025-01-02'},
    {'date': '2025-01-02', 'base': 'EUR', 'stat': 'Open', 'chart_name': 'Chart2', 'query_start': '2025-01-02', 'query_end': '2025-01-03'},
]

df = pd.DataFrame(data)

# 產生圖片的 URL
def generate_image_url(row):
    image_name = f"{row['chart_name']}___{row['base']}___{row['stat']}___{row['query_start']}___{row['query_end']}.png"
    return f"/images/{image_name}"

# 取得所有不重複的圖片
image_urls = set()
for _, row in df.iterrows():
    image_urls.add(generate_image_url(row))

# Dash 頁面 Layout
app.layout = html.Div([
    html.H3("Dash Table"),
    dash_table.DataTable(
        id='table',
        columns=[{"name": col, "id": col} for col in df.columns],
        data=df.to_dict('records')
    ),
    html.H3("對應的圖片"),
    html.Div([html.Img(src=img_url, style={'width': '30%'}) for img_url in image_urls])
])

##
app.run_server(debug=True, port=8050, use_reloader=False)


# 在 Jupyter Notebook 中運行 Flask 應用
from werkzeug.serving import run_simple
run_simple('127.0.0.1', 8050, app.server, use_reloader=False, threaded=True)
# use_reloader=False 防止 Flask 進行自動重啟（這在 Jupyter Notebook 中會有衝突）。
# threaded=True 允許多線程運行，這樣可以避免阻塞主線程

##### serve_image 函數的作用與運作方式

在 Flask（Dash 的後端框架）中，@server.route("/images/<path:image_name>") 是一個路由設定，它告訴 Flask：

* 當使用者請求 /images/xxx.png 時，應該執行 serve_image(image_name) 這個函數。
* 函數會從 IMAGE_FOLDER 內尋找圖片，然後傳送回前端

**為什麼 serve_image 沒有直接被呼叫？**

在 Dash 程式碼裡，你沒有直接執行 serve_image()，而是讓 Flask 自動處理這個請求。這是因為：
1. html.Img(src='/images/sample.png') 這行程式碼，告訴瀏覽器要載入 /images/sample.png。
2. 當瀏覽器請求 /images/sample.png 時，這個請求會被 Flask 處理，觸發 serve_image()。
3. Flask 在 serve_image() 裡面執行 send_from_directory()，找到對應的圖片，並回傳給瀏覽器。

這種方式適合用來提供 assets/ 以外的圖片，例如 ../data_source/ 資料夾內的動態圖片。
你不需要手動呼叫 serve_image()，只要 html.Img(src='/images/xxx.png')，Flask 會自動處理請求。